# Basisdaten und Statistik

Die für dich notwendigen Daten hast du bereits im letzten Teil heruntergeladen.

Aber sind diese Daten für deinen Anwendungszweck überhaupt geeignet? Das zeigt dir eine statistische Auswertung. Das mag langweilig klingen, ist aber für das anschließend folgende Machine Learning enorm wichtig. Sehr viele Projekte scheitern nämlich, weil die Datenqualität einfach nicht stimmt.

Die Datenqualität kannst du auch selbst verbessern, indem du z.B. fehlende Daten erkennst oder solche, bei denen es sich offensichtlich um Fehler (Outlier) handelt. Dieser Vorgang wird oft das *Data Cleansing* bezeichnet. Wenn du Texte verarbeiten möchtest, spielt z.B. hier die Landessprache auch eine große Rolle. Mehr darüber erfährst du im NLP-Kurs.

Häufig wirst du mit zeitabhängigen Daten arbeiten. Diese *Zeitreihen* solltest du genau analysieren, um zu verhindern, dass du mit veralteten Daten arbeitest.

## Statistik des Heise Newstickers

Wie du im letzten Teil gesehen hast, kann man relativ einfach die Daten des Heise Newstickers herunterladen. Wie das genau geht, kannst du im NLP-Videokurs sehen. Dort erkläre ich dir auch, wie man die Texte analysiert und die Anzahl der Wörter (Tokens) bestimmt.

Hier gehen wir davon aus, dass du das schon erledigt hast (oder jemand anders für dich). Die Daten sind schon über die Monate aggregiert und stehen die als CSV-Datei zur Verfügung. Du weißt schon, wie du das mit `pandas` einlesen kannst:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-monthly.csv || wget  https://github.com/heiseacademy/ml-python/raw/main/02-ml-intro/heise-monthly.csv")

In [ ]:
import pandas as pd
heise_monthly = pd.read_csv("heise-monthly.csv", parse_dates=["month"], index_col="month")
heise_monthly

Für jeden Monat findest du hier die Anzahl der Artikel und die mittlere Anzahl an Tokens. Am einfachsten kannst du dir einen `DataFrame` mit `describe` erklären lassen:

In [ ]:
heise_monthly.describe()

Das ist eine sog. [Five Number Summary](https://en.wikipedia.org/wiki/Five-number_summary), in der du wesentliche statistische Eigenschaften findest. Du kannst dir das natürlich auch als Grafik ausgeben lassen, zunächst die Anzahl der Artikel:

In [ ]:
heise_monthly["count"].plot()

Das sieht gut aus, die Anzahl war zwar um 2005/2006 etwas höher, hat sich aber (etwa durch Corona) im Jahr 2020 wieder ziemlich erhöht. Wie sieht es mit der Länge der Artikel in Wörtern (Tokens) aus:

In [ ]:
heise_monthly["tokens"].plot()

Die Heise-Autoren sind wirklich fleißig und schreiben immer längere Texte!

Natürlich haben wir noch nicht die fachliche Domäne überprüft. So könnte es ja sein, dass der Newsticker nur noch Werbung enthält. Die inhaltliche Analyse ist deutlich schwieriger, dazu findest du auch im NLP-Kurs (sehr) viel mehr.

Du kannst an dieser Stelle jedenfalls festhalten, dass die Daten statistisch gut geeignet sind als Basis für *Machine Learning*. Sie sind aktuell, die Textlänge entwickelt sich positiv. Das ist bei weitem nicht immer so!

## Statistik der Konsumindikatoren

In [ ]:
!pip install eurostat

Alle Statistikdaten haben bei Eurostat ein Kürzel. In diesem Beispiel beschäftigst du dich mit den *Konsumindikatoren*, diese tragen das Kürzel `ei_bsco_m`. Es handelt sich dabei um sog. *business and consumer surveys*, die monatlich verdichtet sind (alternativ könntest du die auch auf das Quartal aggregiert herunterladen). Eine genauere Erklärung findest du [hier](https://ec.europa.eu/eurostat/cache/metadata/en/ei_bcs_esms.htm). 

Das Herunterladen der Daten ist nun sehr einfach:

In [ ]:
import eurostat
df = eurostat.get_data_df("ei_bsco_m")

Betrachte im nächsten Schritt die Daten:

In [ ]:
df

`geo\time` ist eine etwas unglückliche Bezeichnung, hier ist nur das Land gemeint, das kannst du einfach umbenennen:

### Datenstruktur anpassen

In [ ]:
df = df.rename(columns={"geo\\TIME_PERIOD": "country"})

Ebenso etwas merkwürdig sind die Monatsnamen in Form von `2021M04` etc., die du auch besser durch ein `datetime`-Objekt in Python ersetzt:

In [ ]:
from datetime import datetime
df.columns = [datetime.strptime(f + "-01", "%Y-%m-%d")
                if f.startswith("20") or f.startswith("19") else f for f in df.columns]

Du interessiert dich nun für die Zeitentwicklung des Konsumindikators *Konsumentenvertrauen* in Deutschland:

### Zeitreihen

In [ ]:
bs_de = df[(df["indic"] == "BS-CSMCI") & (df["s_adj"] == "SA") & (df["country"] == "DE")]
bs_de = bs_de[[c for c in bs_de.columns 
                     if (isinstance(c, datetime) and c.year>=2000)]]
bs_de

Das Ergebnis kannst du plotten und hast dann auch eine Zeitreihe wie oben. Du kannst die Effekte der Finanzkrise und der Corona-Krise sehr gut erkennen. Damit hast du sichergestellt, dass du den richtigen Indikator betrachtest! 

In [ ]:
bs_de.T.plot()

In Deutschland sehen die Indikatoren gut aus, im nächsten Schritt untersuchst du die Vollständigkeit.

### Vollständigkeit der Daten

Dafür betrachtest du die saisonjustierten Daten zu einem beswtimmten Zeitpunkt, allerdings für alle Länder gleichzeitig. Die `pivot`-Funktion kennst du schon, die ist hier sehr nützlich:

In [ ]:
indic_country = df[df["s_adj"] == "SA"].pivot(index="country", columns="indic", values=datetime(2021, 6, 1))
indic_country

In der Tabelle fallen dir schon die `NaN` auf, noch besser kannst du das über eine Heatmap erkennen:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.heatmap(indic_country)

In Rumänien sind für diesen Zeitpunkt gar keine Daten gemeldet worden, ebenso im UK. Hier musst du also mit der Analyse vorsichtig sein. Für den Euro-Raum und die gesamte EU fehlt noch der Indikator `BS-SV-PR`.

### Korrelation der Daten

Im letzten Teil dieses Notebooks betrachtest du die Abhängigkeiten der Indikatoren zueinander. Oftmals wirst du es mit Größen zu tun haben, die gar nicht unabhängig voneinander sind. Um das zu ermitteln, gibt es in Python und `pandas` leistungsfähige Funktionen.

Du schaust dir dazu die Konsumindikatoren für Deutschland ab dem Jahr 2000 an:

In [ ]:
# Jahre ab 2000
columns_2000 = [c for c in df.columns if isinstance(c, datetime) and (c.year>=2000)]

de = df[(df["s_adj"] == "SA") & (df["country"] == "DE")].set_index("indic")[columns_2000].transpose()
de

Die Abhängigkeit jeweils zweier Indikatoren kannst du nun als *Scatterplot* darstellen, am besten geht das mit `seaborn`:

In [ ]:
sns.jointplot(x=de["BS-SFSH"], y=de["BS-CSMCI"], scatter=False, kind="reg")

In der Grafik kannst du erkennen, dass die Indikatoren stark voneinander abhängig sind. Wenn du diese später als Features verwenden willst, musst du sehr aufpassen, dass es dadurch nicht zum sog. *Overfitting* kommt.

Wenn du noch mehr Abhängigkeiten auf einmal darstellen möchtest, kannst du das in einem Gitter anordnen:

In [ ]:
sns.pairplot(de)

Zugebenermaßen sind das ziemlich viele Plots. Bei einigen kannst du eine (positive) Korrelation erkennen, bei anderen eine Antikorrelation. Ein gutes Maß dafür ist das sog. [Pearson-R](https://de.wikipedia.org/wiki/Korrelationskoeffizient#Voraussetzungen_f%C3%BCr_die_Pearson-Korrelation), das du für alle Permutationen aus Indikatoren berechnen kannst.

In [ ]:
from scipy.stats import linregress
from itertools import permutations
res = []
for (i1, i2) in permutations(de.columns, 2):
    res.append({ "Indikator 1": i1, "Indikator 2": i2, "Pearson R": linregress(de[i1], de[i2]).rvalue})

Passenderweise gibt uns `pandas` die mit den stärksten Korrelationen und Antikorrelationen gleich aus:

In [ ]:
pd.DataFrame(res).sort_values("Pearson R")

## Korrelationen nicht unterschätzen

Es ist wichtig, dass du die Korrelationen kennst, damit du nicht mit zu vielen Variablen in deinem Machine Learning-Projekt arbeitest. Sonst besteht die Gefahr des Overfittings.